# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  1.665e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [6]:
np.linalg.matrix_rank(a1)

6

In [7]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [8]:
def check(s):
    N = s.shape[0]
    t = 0
    for i in range(N):
        if np.linalg.det(s[:i + 1, :i + 1]) != 0:
            t += 1
    if t == N:
        return 'условие выполняется'
    else:
        return 'условие не выполняется'

print('Для матрицы a {}'.format(check(a)))
print('Для матрицы a1 {}'.format(check(a1)))


Для матрицы a условие выполняется
Для матрицы a1 условие не выполняется


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [9]:
# Исправленная функция для LU-разложения
def diy_lu_1(a):
    """Construct the LU decomposition of the input matrix.
    
    """
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)  # Матрица перестановок
    for j in range(N-1):
        lam = np.eye(N)
        p_j = np.eye(N)  # Матрица перестановок на каждом новом шаге
        # Поиск индекса ненулевого элелемента в столбце:
        i = np.where(u[j:, j] != 0)[0][0] + j  
        # Формирование матрицы перестановок p_j:
        p_j[i, i], p_j[j, j], p_j[i, j], p_j[j, i] = 0, 0, 1, 1
        u = p_j @ u  # Замена строк в матрице U
        P = p_j @ P
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
        lam[j+1:, j] = gamma
        L = L @ p_j @ lam
    return P @ L, u, P


# Функция, которая по LU-разложению матрицы воссоздаёт саму матрицу
def rec_lu(l, u, p):
    return p @ l @ u


Проверим  на примере матрицы `a`, что новая функция для $LU$-разложения работает:

In [10]:
print('LU-разложение матрицы a:\nМатрица L:\n{}\nМатрица U:\n{}\nМатрица перестановок P:\n{}'.format(*diy_lu_1(a)))

LU-разложение матрицы a:
Матрица L:
[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]]
Матрица U:
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]]
Матрица перестановок P:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


И на примере матрицы `a1`:

In [11]:
print('LU-разложение матрицы a1:\nМатрица L:\n{}\nМатрица U:\n{}\nМатрица перестановок P:\n{}'.format(*diy_lu_1(a1)))

LU-разложение матрицы a1:
Матрица L:
[[ 1.     0.     0.     0.     0.     0.   ]
 [ 1.     1.     0.     0.     0.     0.   ]
 [ 1.     0.     1.     0.     0.     0.   ]
 [ 1.     1.179 -0.09   1.     0.     0.   ]
 [ 1.     1.294 -0.157  1.635  1.     0.   ]
 [ 1.     1.375 -0.208  2.07   2.082  1.   ]]
Матрица U:
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.636e+00 -2.118e+00 -2.348e+00 -2.483e+00 -2.571e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  2.220e-16  0.000e+00  6.143e-02  1.005e-01  1.272e-01]
 [ 0.000e+00 -3.631e-16  0.000e+00  0.000e+00 -1.830e-03 -3.810e-03]
 [ 0.000e+00  2.966e-16  0.000e+00  0.000e+00 -4.337e-19  2.567e-05]]
Матрица перестановок P:
[[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


Теперь проверим, что написанная функция работает правильно. Для этого воспользуемся функцией `rec_lu`, которая по матрицам $L$, $U$ и $P$ восстанавливает первоначальную матрицу. То есть нужно проверить, выполнятся ли равенство `a`$-$ `rec_lu` $(L, U, P)=0$. Сначала сделаем проверку для матрицы `a`:

In [12]:
L, U, P = diy_lu_1(a)
print((a - rec_lu(L, U, P)))

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.220e-16  1.110e-16  1.665e-16]
 [ 0.000e+00  0.000e+00 -2.220e-16  5.551e-17  1.665e-16  1.665e-16]
 [ 0.000e+00  0.000e+00  1.110e-16  1.665e-16 -1.665e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  1.665e-16  1.665e-16 -5.551e-17  0.000e+00]]


И аналогично для `a1`:

In [13]:
L1, U1, P1 = diy_lu_1(a1)
print((a1 - rec_lu(L1, U1, P1)))

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.220e-16  1.110e-16  1.665e-16]
 [ 0.000e+00  0.000e+00 -2.220e-16  5.551e-17  1.665e-16  1.665e-16]
 [ 0.000e+00  0.000e+00  1.110e-16  1.665e-16 -1.665e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  1.665e-16  1.665e-16 -5.551e-17  0.000e+00]]
